In [ ]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
import matplotlib.pyplot as plt
dir_path='/Users/hudsonyeo/Desktop/Python/leo/data/day'
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day')
import time
def process_dir(dir_list):
    x=[]
    for dir_ in dir_list:
        if dir_!='.DS_Store':
            x.append(dir_path+'/'+dir_+'/')
    return x

In [2]:
dir_list_names=os.listdir(dir_path)
dir_list_names.sort()
print(dir_list_names)
dir_list=process_dir(dir_list_names)
dir_list

['.DS_Store', 'RM', 'SR', 'TA', 'c', 'fu', 'm', 'ni', 'zn']


['/Users/hudsonyeo/Desktop/Python/leo/data/day/RM/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/SR/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/c/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/m/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/ni/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/zn/']

In [3]:
def get_file_list(dir_path):
    return os.listdir(dir_path)

class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
class categorise_simple():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        self.threshold.append(np.percentile(array,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)
        return np.asarray(temp)    
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(11)
                elif i<self.threshold[1][1]:
                    temp.append(12)
                elif i<self.threshold[1][2]:
                    temp.append(13)
                elif i<self.threshold[1][3]:
                    temp.append(14)
                elif i<self.threshold[1][4]:
                    temp.append(15)
                elif i<self.threshold[1][5]:
                    temp.append(16)
                elif i<self.threshold[1][6]:
                    temp.append(17)
                elif i<self.threshold[1][7]:
                    temp.append(18)
                elif i<self.threshold[1][8]:
                    temp.append(19)                    
                else:
                    temp.append(20)
            if i<0:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)
                elif i<self.threshold[0][8]:
                    temp.append(9)                    
                else:
                    temp.append(10)
        return np.asarray(temp)    
    
class cross():
    def __init__(self):
        self.time_last_cross=0
        self.current_sign=True
        self.last_time=datetime(1900, 1, 1, 8, 59)
    def get_time(self,time,price):
        if (time-self.last_time)>timedelta(minutes=1):
            self.last_time=time
            self.time_last_cross=time
            return 0
        self.last_time=time
        if (price>0) and self.current_sign : #if price positive and current trend is also positive
            return (time-self.time_last_cross).total_seconds()
        elif (price<0) and (not self.current_sign): #if price negative and current trend is negative
            return (time-self.time_last_cross).total_seconds()
        else: #if price positive, trend negative or price negative, trend positive
            self.time_last_cross=time
            self.current_sign=(price>0)
            return 0    

In [4]:
def calc_smart_price(dataset):
    data=dataset[:]
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_present_vol(dataset):
    data=dataset[:]
    data['current_vol']=data.loc[:,'Volume'].diff().fillna(0)/2
    return data

def calc_future_price(dataset,time_ahead=30,time_index=44, price_col=-2):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,time_index]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),time_index]):
            j+=1
        if (i+j)<length:
            future_price.append(data[(i+j),price_col]) 
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset,future_col=-1,current_col=-3):
    data=dataset.copy()
    temp=data[:,future_col]-data[:,current_col]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset,time_index=44):
    data=dataset[:]
    index=data[:,time_index]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,time_index]=new_index
    return data

def calc_sma_fast(dataset,duration=1,time_index=44,price_col=-4): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,price_col]) #smart price column is -4
    for i in range(len(data)):
        last_time=data[i,time_index]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,time_index]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)

    sma_values=np.asarray(sma_values)
    sma_values=data[:,price_col]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)  

def calc_past_vol(dataset,duration=1,time_index=44,vol_col=-4): #
    data=dataset[:].copy()
    vol_values=[] 
    vol_sum=np.cumsum(data[:,vol_col])
    for i in range(len(data)):
        last_time=data[i,time_index]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,time_index]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) 

def last_cross(dataset,time_index=44,price_col=-3):
    data=dataset[:]
    last_cross=cross()
    timings=[]
    for i in range(len(data)):
        timings.append(last_cross.get_time(data[i,time_index],data[i,price_col]))
    timings=np.asarray(timings)
    timings=np.expand_dims(timings,axis=1)
    return np.concatenate((data,timings),axis=1)

def process(dataset,sma_duration=1,vol_duration=1,time_index=44):
    data=dataset[:]
    data=calc_smart_price(data).values #new
    data=set_index(data,time_index=time_index) #no change
    data=calc_future_price(data,time_index=time_index,price_col=-1) #new
    data=calc_edge(data,future_col=-1,current_col=-2) #new
    data=calc_sma_fast(data,duration=sma_duration,time_index=time_index,price_col=-3) #new
    data=last_cross(data,time_index=time_index,price_col=-1) #new
    return data

In [5]:
def process_files(path,file_list,sma_duration=1,time_index=44):
    df_list=[]
    name_list=[]
    print('processing files...')
    for file in file_list: #read all files and add them to file_list
        if file[-3:]=='csv': #check if file is a CSV
            name_list.append(file)
            df_list.append(process(pd.read_csv(path+file),sma_duration=sma_duration,time_index=time_index))   
            print(file)
    print('complete processing')
    return name_list,df_list

In [9]:
def run_analysis(df_path, df_list,name_list,sma_duration=1):
    error_list=[]
    final_df=pd.DataFrame()    

    for i in range(len(df_list)): #for each 20 day rolling window
        if i<19:
            continue
        print(name_list[i])

        #get -19 day
        sma=df_list[i-19][:,-2].copy() #column for SMA
        time_cross=df_list[i-19][:,-1].copy()

        #get -18 to 0 day (19 days in total)
        for k in range((i-18),i+1): #get 20 day moving averages
            sma=np.concatenate((sma,df_list[k][:,-2].copy()))
            time_cross=np.concatenate((time_cross,df_list[k][:,-1].copy()))

        cat_sma=categorise_10()
        cat_sma.fit(sma) #calculate quartile thresholds for past 20 days

        #get categories for past 20 days
        cat_sma_20=(cat_sma.return_quartile(sma))   

        #get x,y for regression
        x_today=df_list[i][:,-2].copy().astype(float) #column for SMA     
        y_today=df_list[i][:,-3].copy().astype(float) #column for edge
        time_cross_today=df_list[i][:,-1].copy()    

        #removing all NA
        isnum=(~np.isnan(x_today)) & (~np.isnan(y_today))
        #get categories of today's sma  
        cat_x_today=cat_sma.return_quartile(x_today)

        reg_result={}
        reg_result['date']=name_list[i]
        reg_result['total_obs']=len(x_today[isnum])   

        for cat in range(1,21):
            #check past 20 days quartiles and filter time_cross by category
            time_cross_fit=time_cross[cat_sma_20==cat].copy()

            #simple quartile categorisation
            cat_time=categorise_simple()
            cat_time.fit(time_cross_fit)

            #today's sma filter
            sma_filter_today=(cat_x_today==cat)
            #today's time categories
            cat_time_today=cat_time.return_quartile(time_cross_today)


            for quartile in [1,2,3,4]:

                filtered= (isnum & sma_filter_today) #filtering NA and sma quartile
                filtered= filtered & (cat_time_today==quartile) #filtering volume
                new_y=y_today[filtered].copy()

                q='ma_cat_'+str(cat)+'_timecross_quartile'+str(quartile)+'_'
                if (len(new_y)!=0):
                    if np.abs(np.mean(new_y))>10:
                        error_list.append((name_list[i],cat,quartile,new_y))  
                        print('error detected cat',cat,'quartile',quartile)
                    reg_result[(q+'mean')]=np.mean(new_y)
                    reg_result[(q+'std')]=np.std(new_y)
                    reg_result[(q+'num_obs')]=len(new_y)

                else:

                    reg_result[(q+'mean')]=np.nan
                    reg_result[(q+'std')]=np.nan
                    reg_result[(q+'num_obs')]='0'

        final_df=final_df.append(reg_result,ignore_index=True)                
    temp=df_path+'result_10split_time_cross_'+str(sma_duration)+'min_sma_mean.csv'
    final_df.to_csv(temp)
    print('done',temp)   
    return error_list

In [ ]:
#for each path in the directory
for dir_path in dir_list:
    
    file_list=get_file_list(dir_path)
    file_list.sort()
    test_df=pd.read_csv(dir_path+file_list[1])
    
    test_columns=test_df.columns 
    time_index=np.where(test_columns=='Time')[0][0]         
    print('time index',time_index)
    
    for sma_duration in [1,3,5,10]:
        start=time.time()
        print('start',dir_path,sma_duration) 

        name_list,df_list=process_files(dir_path, file_list,sma_duration=sma_duration,time_index=time_index) #process, get SMA and vol data
        mid=time.time()
        print(mid-start)
        #split into categories and get means
        error_list=run_analysis(dir_path+'/results/', df_list,name_list,sma_duration=sma_duration)  
        if len(error_list)!=0:
            print(error_list)
        end=time.time()
        print('complete',end-mid)

In [ ]:
error_list

In [ ]:
dir_path='/Users/hudsonyeo/Desktop/Python/leo/data/day'
dir_list_names=os.listdir(dir_path)
dir_list_names.sort()
dir_list=process_dir(dir_list_names)
#dir_list=dir_list[4:]
for i in range(len(dir_list)):
    dir_list[i]+='results/result_10sma,6vol_1min,'
dir_list,dir_list_names[1:]

In [ ]:
for i in dir_list_names[1:]:
    for num in [1,3,5,10]:
        path='/Users/hudsonyeo/Desktop/Python/leo/data/day/'+i+'/results/result_10sma,6vol_1min,'+str(num)+'min_sma,vol_mean.csv'
        print(i)
        print('sma:',num)
        data=pd.read_csv(path)
        x=[]
        x_std=[]
        pos=[]
        num_obs=[]
        for ma_cat in range(1,21):
            for vol_cat in range(1,5):
                col='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_mean'
                std=data.loc[:,col].std()
                mean=data.loc[:,col].mean()
                column=data.loc[:,col]
                x_std.append(std)
                x.append(mean)
                percentage=len(column[column>=0])/len(column)
                pos.append(percentage)  

                obs='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_num_obs'
                obs=data.loc[:,obs].sum()
                num_obs.append(obs)
        x=np.asarray(x)
        x=np.reshape(x,(20,4))
        x=np.around(x,3)
        x_std=np.asarray(x_std)
        x_std=np.reshape(x_std,(20,4))
        pos=np.asarray(pos)
        pos=np.reshape(pos,(20,4))
        num_obs=np.asarray(num_obs)
        num_obs=np.reshape(num_obs,(20,4))
        print('mean\n',x)
        print('std\n',x_std)
        print('% positive\n',pos)
        print('num obs\n',num_obs)

In [ ]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/results/second/result_10sma,6vol_1min,5min_sma,vol_mean.csv')
x=[]
x_std=[]
pos=[]
num_obs=[]
for ma_cat in range(1,21):
    for vol_cat in range(1,5):
        col='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_mean'
        std=data.loc[:,col].std()
        mean=data.loc[:,col].mean()
        column=data.loc[:,col]
        x_std.append(std)
        x.append(mean)
        percentage=len(column[column>=0])/len(column)
        pos.append(percentage)  
        
        obs='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_num_obs'
        obs=data.loc[:,obs].sum()
        num_obs.append(obs)
x=np.asarray(x)
x=np.reshape(x,(20,4))
x=np.around(x,3)
x_std=np.asarray(x_std)
x_std=np.reshape(x_std,(20,4))
pos=np.asarray(pos)
pos=np.reshape(pos,(20,4))
num_obs=np.asarray(num_obs)
num_obs=np.reshape(num_obs,(20,4))
print('mean\n',x)
print('std\n',x_std)
print('% positive\n',pos)
print('num obs\n',num_obs)

In [11]:
test=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/ni/2019.01.03.csv')

In [16]:
time_index=np.where(test.columns.values=='Time')[0][0]

In [17]:
processed=process(test,time_index=time_index)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
processed[500]

array(['2019-01-03', 1546477449971876, 'A', 'ni1905',
       datetime.datetime(1900, 1, 1, 9, 4, 10, 500000), 86080, 88580,
       86080, 88260, 0, 405076, 0, 35390392960, 332948, 372986, 0, 87170,
       0, 86280, 92400, 81930, 88260, 4, 88270, 3, 20190103, 20190103,
       20190103, 0, 32650000, '192.168.30.96:22', 88265.71428571429,
       88260.76923076923, -4.945054945055745, -2.251073097329936, 0.0],
      dtype=object)